In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import petab.v1.C as C
import pypesto
import pypesto.optimize as optimize
import pypesto.petab
import pypesto.sample as sample
import pypesto.visualize as visualize

import src._petab.equilibrium as eq
from src.utils.pypesto import create_problem_set, load_pypesto_problem
from src.models.sbml import equilibrium
from src.utils.params import ParameterContainer
from src.utils.plot import plot_all_measurements

%load_ext autoreload
%autoreload 2

In [ ]:
pc = ParameterContainer.from_json("/PolyPESTO/src/data/parameters/equilibrium.json")
pg = pc.get_parameter_group("Fast")

paths = create_problem_set(equilibrium, pg, eq.exp_0(), force_compile=True)

In [ ]:
yaml_path = list(paths.values())[0]

importer, problem = load_pypesto_problem(yaml_path, "equilibrium")

In [ ]:
plot_all_measurements(
    importer.petab_problem.measurement_df,
    group_by=C.OBSERVABLE_ID,
    format_axes_kwargs={
        "set_ylim": (0, 1),
    },
    plot_style="both",
    alpha=0.5,
)
plt.show()

In [ ]:
optimizer = optimize.ScipyOptimizer()  # method="Nelder-Mead")
history_options = pypesto.HistoryOptions(trace_record=True)
engine = pypesto.engine.MultiProcessEngine()
n_starts = 1000

result = optimize.minimize(
    problem=problem,
    optimizer=optimizer,
    n_starts=n_starts,
    engine=engine,
    history_options=history_options,
)

In [ ]:
problem.objective.x_names

In [ ]:
sampler = sample.AdaptiveParallelTemperingSampler(
    internal_sampler=sample.AdaptiveMetropolisSampler(),
    n_chains=3,
)

n_samples = 10000
result = sample.sample(
    problem,
    n_samples=n_samples,
    sampler=sampler,
    x0=np.array([2, 2]),
    filename=None,
)
elapsed_time = result.sample_result.time
print(f"Elapsed time: {round(elapsed_time,2)}")

In [ ]:
ax = visualize.sampling_parameter_traces(result, use_problem_bounds=True, size=(12, 5))